### Linear and KNN models

In this notebook I will be creating a knn classification model as well as a linear regression model from our merged datasets. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

In [2]:
df = pd.read_csv("data/merged_dataset.csv")

In [3]:
df.shape

(3133, 54)

Since many datasets were used to create the merged dataframe there will be quite a few missing values that we will have to deal with. 

In [4]:
df.isnull().sum()

county                                                           0
county_fips_code                                                 0
cases                                                            0
deaths                                                           0
stname                                                           0
ctyname                                                          0
tot_pop                                                          0
deaths_per_hun_thou                                              0
cnty_name                                                        1
unemployment_rate                                                1
med_cases_per_100k_change                                        0
med_per_test_results_positive                                   18
transmission_level_high                                          0
transmission_level_low                                           0
transmission_level_moderate                                   

For the columns pertaining to beds and "pct" I will be dropping them. I don't feel that there is a reasonable way to impute these nulls. The columns pertaining to voting are all missing roughly the same rows which probably means that there was no data to collect in those counties. The remaining nulls will be dropped as they are less than 5 percent of the data.  I am also dropping all duplicate columns. Lastly the columns for county, state, and county_fips_code will be dropped as they are identifying columns and will not contribute to our modeling. 

In [6]:
# Dropping columns that I won't be using 
df.drop(columns=['total_beds_7_day_sum', 'cnty_name',
                'ctyname', 'stname', 'state', 'county', 'county_fips_code'], inplace=True)
                

In [7]:
# Dropping remaining nulls 
df.dropna(inplace=True)

In [8]:
df.shape

(2409, 47)

Looking at the chart below there does not appear to be much correlation between our target variable and a lot of our features.  

In [9]:
# Creates correlation matrix for plot
corr_matrix = df.corr()

# Plots correlation of features
plt.figure(figsize=(5, 12))
ax = sns.heatmap(corr_matrix[['deaths_per_hun_thou']]
                 .sort_values(by='deaths_per_hun_thou', ascending=False),
         annot = True,
         vmin = -1,
         vmax = 1,
         cmap = 'winter'
                
                 );

## Modeling 

For my predictions on which counties will need the most help with covid I will be using deaths_per_hun_thou as my y variable. 

### Linear Model

As there is a poor correlation between the feature deaths_per_hun_thou and the rest of our features it does not seem that a linear model is a good fit for our problem statement.

In [10]:
# baseline score 
df['deaths_per_hun_thou'].mean()

315.708127099

In [11]:
# Creates X variable with all columns but deaths_per_hun_thou
X = df[[col for col in df if col not in "deaths_per_hun_thou"]]

# Creates target variable
y = df['deaths_per_hun_thou']

# Instantiating linear regression
linreg = LinearRegression()




# setting cross_val_score
scores = cross_val_score(linreg,
                         X,
                         y,
                         cv=KFold(n_splits=5,
                                  shuffle=True,
                                  random_state=73))

                                                    

In [12]:
scores.mean()

0.2687825176655294

### KNN model

To use our classification model we will need to turn our deaths_per_hun_thou into a binary column. I have chosen to make this column a 1 for counties that are above the 75th percentile of deaths_per_hun_thou and a zero for those that are below. 

In [13]:
df.describe()

cases        deaths       tot_pop  deaths_per_hun_thou  \
count  2.409000e+03   2409.000000  2.409000e+03          2409.000000   
mean   2.561020e+04    319.180158  1.271575e+05           315.708127   
std    7.911837e+04    944.919953  3.682023e+05           137.159741   
min    1.630000e+02      0.000000  1.077000e+03             0.000000   
25%    3.121000e+03     48.000000  1.556100e+04           220.264317   
50%    7.283000e+03    112.000000  3.558900e+04           305.970802   
75%    1.915300e+04    256.000000  9.731100e+04           400.117193   
max    2.312272e+06  28122.000000  1.003911e+07          1102.883252   

       unemployment_rate  med_cases_per_100k_change  \
count        2409.000000                2409.000000   
mean            6.777127                 128.939477   
std             2.191081                  56.354249   
min             1.700000                   0.000000   
25%             5.300000                 100.957000   
50%             6.600000                 131.327000   
75%             8.100000                 164.564000   
max            22.500000                 330.865000   

       med_per_test_results_positive  transmission_level_high  \
count                    2409.000000              2409.000000   
mean                        8.126596                 0.538573   
std                         3.741629                 0.091791   
min                         0.000000                 0.101530   
25%                         5.765000                 0.482615   
50%                         7.780000                 0.539638   
75%                         9.925000                 0.595271   
max                        43.750000                 0.840056   

       transmission_level_low  transmission_level_moderate  ...       SVI__B  \
count             2409.000000                  2409.000000  ...  2409.000000   
mean                 0.166939                     0.155598  ...     0.257368   
std                  0.083982                     0.061195  ...     0.437275   
min                  0.054242                     0.000000  ...     0.000000   
25%                  0.102921                     0.112656  ...     0.000000   
50%                  0.150209                     0.151599  ...     0.000000   
75%                  0.210014                     0.191933  ...     1.000000   
max                  0.625869                     0.403338  ...     1.000000   

            SVI__C       SVI__D      tot_male    tot_female    tot_wa_pop  \
count  2409.000000  2409.000000  2.409000e+03  2.409000e+03  2.409000e+03   
mean      0.249066     0.251142  6.263026e+04  6.452722e+04  9.759208e+04   
std       0.432562     0.433760  1.811967e+05  1.870413e+05  2.643150e+05   
min       0.000000     0.000000  5.440000e+02  5.330000e+02  9.330000e+02   
25%       0.000000     0.000000  7.813000e+03  7.675000e+03  1.262900e+04   
50%       0.000000     0.000000  1.766300e+04  1.766500e+04  3.064400e+04   
75%       0.000000     1.000000  4.788300e+04  4.908700e+04  8.033200e+04   
max       1.000000     1.000000  4.949041e+06  5.090066e+06  7.098309e+06   

         tot_ba_pop    tot_aa_pop    tot_na_pop  minority_group_total  
count  2.409000e+03  2.409000e+03   2409.000000          2.409000e+03  
mean   1.667754e+04  7.444982e+03    318.605645          1.667754e+04  
std    6.218002e+04  4.794595e+04   2479.081403          6.218002e+04  
min    2.000000e+00  0.000000e+00      0.000000          2.000000e+00  
25%    2.140000e+02  8.900000e+01      7.000000          2.140000e+02  
50%    1.289000e+03  2.730000e+02     24.000000          1.289000e+03  
75%    8.500000e+03  1.394000e+03    104.000000          8.500000e+03  
max    1.226489e+06  1.545445e+06  93325.000000          1.226489e+06  

[8 rows x 47 columns]

In [14]:
# Creates binary column
df.deaths_per_hun_thou = np.where(df['deaths_per_hun_thou'] >= 405, 1, 0)

In [15]:
# baseline
1 - df.deaths_per_hun_thou.mean()

0.7633872976338729

In [16]:
# Sets up X variable for columns that have not been dropped
X = df[[col for col in df if col not in "deaths_per_hun_thou"]]

# Sets up target variable
y = df['deaths_per_hun_thou']


# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state = 73,
                                                    stratify = y) 


# Instantiate a StandardScaler + kNN pipeline.
pipe_knn = Pipeline([
    ('ss', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# Define dictionary of hyperparameters.
pipe_knn_params = {
    'ss__with_mean' : [True, False],
    'ss__with_std' : [True, False],
    'knn__weights' : ['uniform', 'distance'],
    'knn__n_neighbors' : [1, 3, 5, 10, 50],
    'knn__p' : [1, 2]
}




# Instantiate our GridSearchCV object.
pipe_knn_gridsearch = GridSearchCV(pipe_knn, 
                                 pipe_knn_params,
                                 cv=5,
                                 verbose=1)
# Fitting gridsearch
pipe_knn_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [1, 3, 5, 10, 50],
                         'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance'],
                         'ss__with_mean': [True, False],
                         'ss__with_std': [True, False]},
             verbose=1)

In [17]:
X.shape

(2409, 45)

In [18]:
pipe_knn_gridsearch.best_params_

{'knn__n_neighbors': 50,
 'knn__p': 1,
 'knn__weights': 'distance',
 'ss__with_mean': True,
 'ss__with_std': True}

In [19]:
pipe_knn_gridsearch.score(X_test, y_test)

0.7946058091286307

Our knn model has performed much better than our linear model with a .783 percent accuracy. Given that the classification model has performed much better I am going to further adjust the features that I'm using for my X variable to see if I can gain more meaningful results. To start I'm going to drop the population columns as we can naturaly infer that a higher population will of course see higher death rates. I will also be dropping the voting and minority_group_total columns as they have a low correlation with my target variable. 

In [20]:
# correlation of populations
df[[col for col in df if col.startswith("tot_")]].corr()

tot_pop  tot_male  tot_female  tot_wa_pop  tot_ba_pop  \
tot_pop     1.000000  0.999900    0.999906    0.990255    0.796024   
tot_male    0.999900  1.000000    0.999612    0.990757    0.790400   
tot_female  0.999906  0.999612    1.000000    0.989580    0.801320   
tot_wa_pop  0.990255  0.990757    0.989580    1.000000    0.733440   
tot_ba_pop  0.796024  0.790400    0.801320    0.733440    1.000000   
tot_aa_pop  0.838057  0.840743    0.835294    0.798099    0.538702   
tot_na_pop  0.474010  0.477487    0.470552    0.439266    0.244864   

            tot_aa_pop  tot_na_pop  
tot_pop       0.838057    0.474010  
tot_male      0.840743    0.477487  
tot_female    0.835294    0.470552  
tot_wa_pop    0.798099    0.439266  
tot_ba_pop    0.538702    0.244864  
tot_aa_pop    1.000000    0.597579  
tot_na_pop    0.597579    1.000000

In [21]:
# dropping population columns
df.drop(columns=[col for col in df if col.startswith("tot_")], inplace=True)


In [22]:
# dropping voting and minority_group_total
df.drop(columns=[col for col in df if col.endswith('votes') or col.startswith('minority')], inplace=True)


In [23]:
# Sets up X variable for columns that have not been dropped
X = df[[col for col in df if col not in "deaths_per_hun_thou"]]

# Sets up target variable
y = df['deaths_per_hun_thou']

# sets up train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state = 73,
                                                    stratify = y) 




# Instantiate our GridSearchCV object.
pipe_knn_gridsearch_2 = GridSearchCV(pipe_knn, 
                                 pipe_knn_params,
                                 cv=5,
                                 verbose=1)

# Fitting gridsearch
pipe_knn_gridsearch_2.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [1, 3, 5, 10, 50],
                         'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance'],
                         'ss__with_mean': [True, False],
                         'ss__with_std': [True, False]},
             verbose=1)

In [24]:
pipe_knn_gridsearch_2.score(X_test, y_test)

0.7904564315352697

In [25]:
pipe_knn_gridsearch_2.best_params_

{'knn__n_neighbors': 50,
 'knn__p': 1,
 'knn__weights': 'distance',
 'ss__with_mean': True,
 'ss__with_std': True}

### Conclusion and Recommendations

Although the KNN model provided a score of .783 accuracy this problem seems to be best answered through standard EDA and inference. However in the future to further improve upon the model I would recommend gathering more recent data when available and looking specifically into the hospitals available in each county.(Staffing, building size, occupancy)

In [26]:
X.shape

(2409, 34)